In [118]:
import numpy as np
import pandas as pd
import pickle 

In [154]:
df = pickle.load(open('dataset_level2.pkl','rb'))
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,2,Australia,Sri Lanka,0.1,0,0,NaN,Melbourne Cricket Ground
1,2,Australia,Sri Lanka,0.2,0,0,NaN,Melbourne Cricket Ground
2,2,Australia,Sri Lanka,0.3,1,0,NaN,Melbourne Cricket Ground
3,2,Australia,Sri Lanka,0.4,2,0,NaN,Melbourne Cricket Ground
4,2,Australia,Sri Lanka,0.5,0,0,NaN,Melbourne Cricket Ground


In [155]:
df.isnull().sum()

match_id               0
batting_team           0
bowling_team           0
ball                   0
runs                   0
player_dismissed       0
city                8549
venue                  0
dtype: int64

In [156]:
df[df['city'].isnull()]['venue'].value_counts()

venue
Dubai International Cricket Stadium        3092
Pallekele International Cricket Stadium    2066
Melbourne Cricket Ground                   1453
Sydney Cricket Ground                       749
Adelaide Oval                               498
Harare Sports Club                          372
Sylhet International Cricket Stadium        128
Sharjah Cricket Stadium                     127
Carrara Oval                                 64
Name: count, dtype: int64

In [157]:
cities = np.where(df['city'].isnull(),df['venue'].str.split().apply(lambda x:x[0]),df['city'])
df['city'] = cities

In [158]:
df.isnull().sum()

match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

In [159]:
df.drop(columns=['venue'],inplace=True)
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne


In [160]:
eligible_cities = df['city'].value_counts()[df['city'].value_counts() > 600].index.tolist()
df = df[df['city'].isin(eligible_cities)]

In [161]:
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne


In [162]:
print(df.dtypes)

match_id              int64
batting_team         object
bowling_team         object
ball                float64
runs                  int64
player_dismissed     object
city                 object
dtype: object


In [163]:
df['current_score'] = df.groupby('match_id')['runs'].cumsum()

In [164]:
df['over'] = df['ball'].apply(lambda x:str(x).split(".")[0])
df['ball_no'] = df['ball'].apply(lambda x:str(x).split(".")[1])
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5
...,...,...,...,...,...,...,...,...,...,...
312869,2601,Sri Lanka,Australia,19.3,1,0,Colombo,125,19,3
312870,2601,Sri Lanka,Australia,19.4,0,0,Colombo,125,19,4
312871,2601,Sri Lanka,Australia,19.5,0,DM de Silva,Colombo,125,19,5
312872,2601,Sri Lanka,Australia,19.6,2,0,Colombo,127,19,6


In [165]:
df['balls_bowled'] = (df['over'].astype('int')*6) + df['ball_no'].astype('int')
df['balls_left'] = 120 - df['balls_bowled']
df['balls_left'] = df['balls_left'].apply(lambda x:0 if x<0 else x)
df['player_dismissed'] = df['player_dismissed'].apply(lambda x:0 if x=='0' else 1)
df['player_dismissed'] = df['player_dismissed'].astype('int')
df['player_dismissed'] = df.groupby('match_id')['player_dismissed'].cumsum()
df['wickets_left'] = 10 - df['player_dismissed']
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,1,119,10
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,2,118,10
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,3,117,10
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,4,116,10
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,5,115,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
312869,2601,Sri Lanka,Australia,19.3,1,8,Colombo,125,19,3,117,3,2
312870,2601,Sri Lanka,Australia,19.4,0,8,Colombo,125,19,4,118,2,2
312871,2601,Sri Lanka,Australia,19.5,0,9,Colombo,125,19,5,119,1,1
312872,2601,Sri Lanka,Australia,19.6,2,9,Colombo,127,19,6,120,0,1


In [166]:
df['crr'] = (df['current_score']*6)/df['balls_bowled']
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,0,0,1,1,119,10,0.000000
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,0,0,2,2,118,10,0.000000
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,1,0,3,3,117,10,2.000000
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,3,0,4,4,116,10,4.500000
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,3,0,5,5,115,10,3.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312869,2601,Sri Lanka,Australia,19.3,1,8,Colombo,125,19,3,117,3,2,6.410256
312870,2601,Sri Lanka,Australia,19.4,0,8,Colombo,125,19,4,118,2,2,6.355932
312871,2601,Sri Lanka,Australia,19.5,0,9,Colombo,125,19,5,119,1,1,6.302521
312872,2601,Sri Lanka,Australia,19.6,2,9,Colombo,127,19,6,120,0,1,6.350000


In [167]:
groups = df.groupby('match_id')

match_ids = df['match_id'].unique()
last_five = []

for id in match_ids:
    runs_rolling = groups.get_group(id)['runs'].rolling(window=30).sum()
    last_five.extend(runs_rolling.values.tolist())

df['last_five'] = last_five


In [168]:
final_df = df.groupby('match_id').sum()['runs'].reset_index().merge(df,on='match_id')

In [169]:
final_df=final_df[['batting_team','bowling_team','city','current_score','balls_left','wickets_left','crr','last_five','runs_x']]

In [170]:
final_df.dropna(inplace=True)

In [171]:
final_df.isnull().sum()

batting_team     0
bowling_team     0
city             0
current_score    0
balls_left       0
wickets_left     0
crr              0
last_five        0
runs_x           0
dtype: int64

In [172]:
final_df = final_df.sample(final_df.shape[0])
final_df.sample(2)

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
6617,Bangladesh,India,Colombo,98,39,6,7.259259,37.0,166
2023,England,Australia,Hobart,49,91,9,10.137931,49.0,155


In [75]:
print(final_df.dtypes)

match_id           int64
batting_team      object
bowling_team      object
runs_x             int64
current_score      int64
wickets_left       int32
crr              float64
city              object
runs_y             int64
balls_left         int64
dtype: object


In [173]:
final_df=final_df[['batting_team','bowling_team','city','current_score','balls_left','wickets_left','crr','last_five','runs_x']]

In [174]:
final_df.head()

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
59064,Pakistan,Australia,Birmingham,37,88,9,6.937500,37.0,162
14730,Pakistan,England,Manchester,64,73,8,8.170213,38.0,190
76889,South Africa,Bangladesh,Mirpur,68,75,10,9.066667,41.0,169
32500,New Zealand,Pakistan,Christchurch,143,5,3,7.460870,45.0,147
72163,New Zealand,Bangladesh,Mirpur,79,68,9,9.115385,37.0,204


In [175]:
pickle.dump(final_df,open('dataset_final.pkl','wb'))